In [1]:
import gensim
from gensim import utils
import numpy as np
import sys
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
%matplotlib inline

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [3]:
train = pd.read_csv("./NLP_data/master_train.csv",index_col=0)
val = pd.read_csv("./NLP_data/master_val.csv",index_col=0)
test = pd.read_csv("./NLP_data/master_test.csv",index_col=0)

In [4]:
print(train.info())
print(val.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123034 entries, 0 to 123033
Data columns (total 6 columns):
image               123034 non-null int64
phrase1             123034 non-null object
phrase2             123034 non-null object
original_phrase1    123034 non-null object
original_phrase2    123034 non-null object
ytrue               123034 non-null bool
dtypes: bool(1), int64(1), object(4)
memory usage: 5.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4078 entries, 0 to 4077
Data columns (total 6 columns):
image               4078 non-null int64
phrase1             4078 non-null object
phrase2             4078 non-null object
original_phrase1    4078 non-null object
original_phrase2    4078 non-null object
ytrue               4078 non-null bool
dtypes: bool(1), int64(1), object(4)
memory usage: 195.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4526 entries, 0 to 4525
Data columns (total 6 columns):
image               4526 non-null int64
phrase1   

In [15]:
def preprocess(text):
    text = str(text)
    text = text.lower()
    doc = text.split("+")
    return doc

In [6]:
def list2vec(l):
    if len(l)==0:
        nanvec = np.empty((300,), float)
        nanvec.fill(np.nan)
        return nanvec
    nl = [model[w] if w in model.vocab else np.nan for w in l]
    return np.nanmean(nl,axis=0)

In [29]:
fu_train = open("train_uniquePhrases_master","w")
fu_val = open("val_uniquePhrases_master","w")
fu_test = open("test_uniquePhrases_master","w")

In [30]:
phrase_train=list(set(train.phrase1.tolist()+train.phrase2.tolist()))
phrase_test=list(set(test.phrase1.tolist()+test.phrase2.tolist()))
phrase_val=list(set(val.phrase1.tolist()+val.phrase2.tolist()))

In [31]:
print(len(phrase_train))
print(len(phrase_val))
print(len(phrase_test))

49574
1988
2076


In [32]:
for phrase in phrase_train:
    fu_train.write(phrase+"\n")

In [33]:
for phrase in phrase_test:
    fu_test.write(phrase+"\n")

In [34]:
for phrase in phrase_val:
    fu_val.write(phrase+"\n")

In [35]:
fu_train.close()
fu_test.close()
fu_val.close()

In [36]:
l2v_train = [list2vec(preprocess(phrase)) for phrase in phrase_train]
l2v_test = [list2vec(preprocess(phrase)) for phrase in phrase_test]
l2v_val = [list2vec(preprocess(phrase)) for phrase in phrase_val]

In [39]:
l2v_train = np.array(l2v_train)
l2v_val = np.array(l2v_val)
l2v_test = np.array(l2v_test)

In [41]:
print(l2v_train.shape)
print(l2v_val.shape)
print(l2v_test.shape)

(49574, 300)
(1988, 300)
(2076, 300)


In [42]:
np.save('train_master.npy', l2v_train)
np.save('test_master.npy', l2v_test)
np.save('val_master.npy', l2v_val)